In [1]:
using ITensors,ITensorMPS
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

In [3]:
include("overload_Spin=3h.jl")

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for j=1:N-1
        ops += J,   "Sz",j, "Sz",j+1
        ops += J/2, "S+",j, "S-",j+1
        ops += J/2, "S-",j, "S+",j+1
    end
    #make it a ring
#    ops += J,   "Sz",N, "Sz",1
#    ops += J/2, "S+",N, "S-",1
#    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for j=1:N
        ops += -B, "Sz",j
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:
function cmag(wf::MPS,j_list)    
    psi = copy(wf)
    sites = siteinds(psi)

    for j in j_list
        if isodd(j)
            psi = apply( op("S+",sites[j]) * op("S-",sites[j+1]), psi )
        else
            psi = apply( op("S-",sites[j]) * op("S+",sites[j+1]), psi )
        end    
    end

    #----------------------------------
    return normalize!(psi)
end

cmag (generic function with 1 method)

In [6]:
# one or two sites reduced density matrix
#--------------------------------------
function get_2pdm(wf::MPS,locs)
    ket = copy(wf)
    sites = siteinds(ket)
    n_locs = length(locs)

    if n_locs==1
        A = locs[1]
        orthogonalize!(ket,A)
        rho = prime(ket[A],sites[A]) * dag(ket[A])

    elseif n_locs==2
        A,B = locs
        orthogonalize!(ket,A)
        bra = prime(dag(ket),linkinds(ket))
        
        rho = prime(ket[A],linkinds(ket,A-1)) * prime(bra[A],sites[A])
        [ rho *= ket[j]*bra[j] for j in A+1:B-1 ]
        rho *= prime(ket[B],linkinds(ket,B)) * prime(bra[B],sites[B])  
    
    end

    #-----------------------------------
    return rho
end

get_2pdm (generic function with 1 method)

In [7]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log2(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [8]:
#   entanglement details between two sites
#-----------------------------------
function calc_Neg(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho_AB = get_2pdm(psi,[A,B])
    rho_AB_PT = calc_PartialTranspose(rho_AB)
    egn_val = calc_Eigenvalues(rho_AB_PT)

    neg_entries =  [ lam for lam in egn_val if lam<0 ]
    Neg = abs(sum(neg_entries))

    #-----------------------------------
    return Neg
end


# Partial between two sites (test program)
#   takes a partial transpose over 2nd site
#------------------------------------------------

function calc_PartialTranspose(density_matrix)
    rho = copy(density_matrix)

    oldinds = inds(rho)
    newinds = [oldinds[1],oldinds[2],oldinds[4],oldinds[3]]
    rho_PT = swapinds(rho,oldinds,newinds)
    
    #-----------------------------
    return rho_PT
end


#   entanglement details between two sites
#-----------------------------------
function calc_MutInf(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho_A  = get_2pdm(psi,[A,A])
    rho_B  = get_2pdm(psi,[B,B])
    rho_AB = get_2pdm(psi,[A,B])

    S_A = calc_SvN(rho_A)
    S_B = calc_SvN(rho_B)
    S_AB = calc_SvN(rho_AB)

    MutInf = S_A + S_B - S_AB

    #-----------------------------------
    return MutInf
end




calc_MutInf (generic function with 1 method)

In [9]:
#define the parameters of the hamiltonian
#----------------------------------------------------------------------------

J = 1
B = 0.1

nsweeps = 11
maxdim = [10,25,50,100,150,200,250]
cutoff = [1E-10]

1-element Vector{Float64}:
 1.0e-10

In [10]:
N = 20
#--------------------------------------------

site_list = collect(1:N) 
sites = siteinds( j->isodd(j) ? "S=1/2" : "S=3/2", N ) 

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true)
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-19.534428139075185  maxlinkdim=8 maxerr=1.18E-15 time=28.153
After sweep 2 energy=-19.918876101341677  maxlinkdim=25 maxerr=7.59E-09 time=0.064
After sweep 3 energy=-19.934293879818963  maxlinkdim=49 maxerr=9.81E-11 time=0.504
After sweep 4 energy=-19.935366695502935  maxlinkdim=48 maxerr=9.77E-11 time=1.328
After sweep 5 energy=-19.935477855907507  maxlinkdim=33 maxerr=9.91E-11 time=0.443
After sweep 6 energy=-19.935488852513085  maxlinkdim=22 maxerr=9.95E-11 time=0.306
After sweep 7 energy=-19.93548961582521  maxlinkdim=17 maxerr=9.96E-11 time=0.096
After sweep 8 energy=-19.93548967490369  maxlinkdim=14 maxerr=9.84E-11 time=0.196
After sweep 9 energy=-19.935489677747235  maxlinkdim=14 maxerr=9.61E-11 time=0.118
After sweep 10 energy=-19.935489677937703  maxlinkdim=14 maxerr=9.80E-11 time=0.057
After sweep 11 energy=-19.93548967811057  maxlinkdim=14 maxerr=9.12E-11 time=0.071


(-19.93548967811057, MPS
[1] ((dim=2|id=551|"Link,l=1"), (dim=2|id=811|"S=1/2,Site,n=1"))
[2] ((dim=7|id=537|"Link,l=2"), (dim=4|id=928|"S=3/2,Site,n=2"), (dim=2|id=551|"Link,l=1"))
[3] ((dim=2|id=214|"S=1/2,Site,n=3"), (dim=11|id=612|"Link,l=3"), (dim=7|id=537|"Link,l=2"))
[4] ((dim=4|id=851|"S=3/2,Site,n=4"), (dim=14|id=289|"Link,l=4"), (dim=11|id=612|"Link,l=3"))
[5] ((dim=2|id=472|"S=1/2,Site,n=5"), (dim=13|id=944|"Link,l=5"), (dim=14|id=289|"Link,l=4"))
[6] ((dim=4|id=716|"S=3/2,Site,n=6"), (dim=14|id=308|"Link,l=6"), (dim=13|id=944|"Link,l=5"))
[7] ((dim=2|id=897|"S=1/2,Site,n=7"), (dim=12|id=250|"Link,l=7"), (dim=14|id=308|"Link,l=6"))
[8] ((dim=4|id=787|"S=3/2,Site,n=8"), (dim=13|id=375|"Link,l=8"), (dim=12|id=250|"Link,l=7"))
[9] ((dim=2|id=264|"S=1/2,Site,n=9"), (dim=12|id=442|"Link,l=9"), (dim=13|id=375|"Link,l=8"))
[10] ((dim=4|id=335|"S=3/2,Site,n=10"), (dim=12|id=407|"Link,l=10"), (dim=12|id=442|"Link,l=9"))
[11] ((dim=2|id=828|"S=1/2,Site,n=11"), (dim=12|id=807|"Link,l=1

In [11]:
exp_H = inner(psi0',H,psi0) ;
exp_Sz = expect( psi0, "Sz" ) ;
net_Sz = sum(exp_Sz) ;

println("energy/N = $(exp_H/N)")
println("net(Sz)/N = $(net_Sz/N)")

states = [ isodd(j) ? "Dn" : "+3/2" for j in 1:N]
psi_Neel = productMPS(sites,states)
alpha_Neel = inner(psi0,psi_Neel)
#alpha_Neel = 0.99053*exp(-0.03547*N)

energy/N = -0.9967744839055293
net(Sz)/N = 0.49999999997516625


0.4528309491247689

In [12]:
rho = get_4pdm(psi0,[2,6])
#inds(rho)

#rho = get_2pdm(psi0,[2,7])
#wcalc_properties(rho)

LoadError: UndefVarError: `get_4pdm` not defined

In [13]:
psi = cmag(psi0,[3,5,7,9,13,15,17])

plot( expect(psi,"Sz"),marker=:circle )

GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0
